In [1]:
!pip install pygmo

In [2]:
!pip install pymoo

In [3]:
from numpy.random import randint
from numpy.random import rand
import numpy as np
import random

# Generate MTSP

In [4]:
n_cities = 3
grid_size = 100
n_objectives = 3
coordinates = []
edges_map = []


In [5]:
# Generate a multi-objective TSP map and save to csv
from scipy.spatial.distance import pdist, squareform, cdist
output_path = 'test_map/'
for i in range(n_objectives):
  random_city_coordinate = np.random.random((n_cities, 2)) * [grid_size, grid_size]
  coordinates.append(random_city_coordinate)
  edges_map.append(cdist(random_city_coordinate, random_city_coordinate))
  #np.savetxt(output_path+'coords_'+str(n_cities)+'cities_obj_'+str(i)+'.csv', coordinates[i], delimiter=",")
  #np.savetxt(output_path+'value_'+str(n_cities)+'cities_obj_'+str(i)+'.csv', edges_map[i], delimiter=",")

In [6]:
# Read a file
#for i in range(n_objectives):
#  edges_map.append(np.genfromtxt(output_path+'value_'+str(n_cities)+'cities_obj_'+str(i)+'.csv', delimiter=","))

# Sub functions

## Minimax Regret Function

In [7]:
def objective(x):
    n_cities = len(x)
    objectives = []
    for o in range(n_objectives):
      dist = 0
      for k in range(n_cities - 1):
          i, j = x[k], x[k + 1]
          dist += edges_map[o][i, j]
      objectives.append(dist)
    return tuple(objectives)
    
def scalarizer(weights, decision):
    return sum([w*x for w, x in zip(weights,decision)])

def pairwise_max_regret(omega_set, x1,x2):
    pairwise_regret = []
    for w in omega_set:
      regret = scalarizer(w, objective(x1)) - scalarizer(w, objective(x2))
      pairwise_regret.append(regret)
    return np.max(pairwise_regret), np.argmax(pairwise_regret)
    
def minimax_regret(omega_set, X_pop):
    PMR = np.zeros(shape=(len(X_pop),len(X_pop)))
    np.fill_diagonal(PMR, -np.inf)
    PMR_w = np.zeros(shape=(len(X_pop),len(X_pop)))
    for i in range(0,len(X_pop)):
        for j in range(0,len(X_pop)):
            if i != j:
                PMR[i][j], PMR_w[i][j] = pairwise_max_regret(omega_set, X_pop[i], X_pop[j])
    MR = PMR.max(axis=1)
    argmax_index = PMR.argmax(axis=0)[0]
    MMR = MR.min()
    argmin_index = MR.argmin()
    argmin_MMR = X_pop[argmin_index]
    argmin_MMR_w = omega_set[int(PMR_w[argmin_index,argmax_index])]
    print('MMR : ', MMR, 'X* : ',argmin_MMR)
    print('W* : ', argmin_MMR_w)
    return MMR, argmin_MMR, argmin_MMR_w

## Genetic Algorithm Operators

In [8]:
import math

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
  c = p1
    # check for recombination
  if rand() < r_cross:
    pt = randint(0, len(p1))
    if pt == 0:
      c = p2
    if pt == len(p1):
      c = p1
    else:
      c = p1[:pt] + p2[pt:]
  return c

def solution_crossover(p, r_cross):
	# children are copies of parents by default
  c = p
    # check for recombination
  if rand() < r_cross:
    if len(p) < 3:
      return p
    elif len(p) == 3:
      c = [p[0],p[2],p[1]]
    else:
      pt = randint(2, len(p)-1)
      c = [p[0]] + p[pt:] + p[1:pt]
  return c

# mutation operator
def weight_mutation(string, r_mut):
  string = list(string)
  for i in range(len(string)):
    # check for a mutation
    if rand() < r_mut:
    # flip the bit
      string[i] = 1-string[i]
  # normalize
  if sum(string) > 0:
    string = [s/sum(string) for s in string]
  return tuple(string)

def solution_mutation(string, r_mut):
  string = list(string)
  for i in range(len(string)):
    # check for a mutation
    if rand() < r_mut/2:
    # flip the bit
      idx = range(len(string))
      i1, i2 = random.sample(idx, 2)
      string[i1], string[i2] = string[i2], string[i1]
  # normalize
  return string

def crossover_mutation(omega_set, n_children, r_cross=1.0, r_mut=1.0, c_type = 'weight'):
    children = []
    for i in range(0, n_children):
        [w1, w2] = random.sample(omega_set,2)
        if c_type == 'weight':
          c = crossover(w1, w2, r_cross)
          m = weight_mutation(c, r_mut)
        if c_type == 'solution':
          c = solution_crossover(w1,r_cross)
          m = solution_mutation(c, r_mut)
        children.append(m) 
    return children

def get_tuple_distance(x,y):
    return sum([(a-b)**2 for a, b in zip(x,y)])/len(x)

def selection(x, pop, k):
    dist_array = [get_tuple_distance(x,p) for (w,p) in pop]
    sort_index = np.argsort(dist_array).tolist()
    selected_pop = [pop[i] for i in sort_index[0:k]]
    selected_x = [x for (_,x) in selected_pop]
    return selected_x, selected_pop


## NSGA-II : Existing MTSP Solver from Pymoo



In [9]:
import autograd.numpy as anp
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform, cdist

from pymoo.model.problem import Problem


class MTSP(Problem):

    def __init__(self, cities_map, weights,**kwargs):
        """
        A two-dimensional traveling salesman problem (TSP)
        Parameters
        ----------
        cities : numpy.array
            The cities with 2-dimensional coordinates provided by a matrix where where city is represented by a row.
        """
        n_cities, _ = cities_map[0].shape
        self.cities_map = cities_map
        self.weights = weights
        super(MTSP, self).__init__(
            n_var=n_cities,
            n_obj=3,
            xl=0,
            xu=n_cities,
            type_var=int,
            elementwise_evaluation=True,
            **kwargs
        )

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = self.weights[0]*self.get_route_length(x, self.cities_map[0])
        f2 = self.weights[1]*self.get_route_length(x, self.cities_map[1])
        f3 = self.weights[2]*self.get_route_length(x, self.cities_map[2])
        out['F'] = anp.column_stack([f1, f2, f3])

    def get_route_length(self, x, D):
        n_cities = len(x)
        dist = 0
        for k in range(n_cities - 1):
            i, j = x[k], x[k + 1]
            dist += D[i, j]
        last, first = x[-1], x[0]
        dist += D[last, first]  # back to the initial city
        return dist


In [10]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling
from pymoo.util.termination.default import SingleObjectiveDefaultTermination
from pymoo.model.repair import Repair

class StartFromZeroRepair(Repair):

    def _do(self, problem, pop, **kwargs):
        X = pop.get("X")
        I = np.where(X == 0)[1]

        for k in range(len(X)):
            i = I[k]
            x = X[k]
            _x = np.concatenate([x[i:], x[:i]])
            pop[k].set("X", _x)

        return pop

def find_optimal_result(weights, map):
    problem = MTSP(map, weights)

    algorithm = NSGA2(
        pop_size=20,
        sampling=get_sampling("perm_random"),
        crossover=get_crossover("perm_erx"),
        mutation=get_mutation("perm_inv"),
        repair=StartFromZeroRepair(),
        eliminate_duplicates=True
    )

    termination = SingleObjectiveDefaultTermination(n_last=10, n_max_gen=np.inf)

    res = minimize(
        problem,
        algorithm,
        termination,
        seed=1,
    )
    return res

def find_optimal_x(weights, map):
    result = find_optimal_result(weights, map)
    return result.X[0]

# RIGA Algorithm

In [11]:
#Initialize weights and linear constraints
def DecimalToBinary(num, n_bits): 
    binary = []
    while num >= 1:
      binary.append(num % 2)
      num = num//2
    if len(binary) < n_bits:
      binary = binary+([0]*(n_bits-len(binary)))
    binary = binary[::-1]
    return tuple(binary)
def polyhedral(n_objectives):
    result = []
    for i in range(0, n_objectives):
      temp = [0]*n_objectives
      temp[i] = 1
      result.append(temp)
    return result
def initialize_linear_constraints(omega_set):
  extreme_points = [(w1,w2) for [w1,w2,_] in omega_set]
  lines = []
  for i in range(len(extreme_points)-1):
    for j in range(i+1, len(extreme_points)):
      lines.append(LineString([extreme_points[i],extreme_points[j]]))
  return lines

In [12]:
from shapely.geometry import LineString, Polygon, Point
import time

def query(pop, theta_x):
    c = 0
    t = 1000
    while True:
      (w1, x1), (w2, x2) = random.sample(pop,2)
      if (x1, x2) not in theta_x and (x2, x1) not in theta_x and x1 != x2:
        break
      if c >= t:
        return (None, None), (None, None)
      c+=1
    #pop_non_dup = [p for p in pop if p[0] not in big_o_x]
    return (w1, x1), (w2, x2)
# omega_linear_constraints = [[coefficient of w1,w2,w3, bias]]
# cw1 * w1 + cw2 * w2 + ... + cwn * wn + bias <= 0
# only allow 3 objective max

def ask_DM(x1,x2,w_star):
    y1 = objective(x1)
    y2 = objective(x2)
    fyx1 = scalarizer(w_star, y1)
    fyx2 = scalarizer(w_star, y2)
    if fyx1 <= fyx2:
      return x1, x2, y1, y2
    else:
      return x2, x1, y2, y1

def solve_omega_set(a,b,omega_set,lines):
    cw1 = a[0] - b[0] - (a[2] - b[2])
    cw2 = a[1] - b[2] - (a[2] - b[2])
    bias = a[2] - b[2]
    if cw1 == 0 or cw2 == 0:
      return omega_set, lines 
      
    p1 = (-bias/cw1,0)
    p2 = (0,-bias/cw2)
    lines.append(LineString([p1, p2]))
    intersect = []
    for i in range(len(lines)-1):
      for j in range(i+1, len(lines)):
        intersect_point = lines[i].intersection(lines[j])
        if type(intersect_point) == Point and intersect_point.coords != []:
          intersect.append((intersect_point.x,intersect_point.y))
    polygon = Polygon(intersect)
    while True:
      random_coord_x, random_coord_y = random.random(),random.random()
      if polygon.contains(Point(random_coord_x, random_coord_y)):
        break
    new_omega = [random_coord_x, random_coord_y, 1-(random_coord_x+random_coord_y)]
    omega_set.append(new_omega)
    return omega_set, lines

def riga_search(map, threshold = 0, n_objectives = 3, s=4, k=2, m=2, no_improvement_threshold=20):
    start_time = time.time()
    dm_counter = 0
    theta_x = []
    omega_set = polyhedral(n_objectives)
    linear_constraints = initialize_linear_constraints(omega_set)
    pop = [] 
    for w in omega_set:
      x = find_optimal_x(w,map).tolist()
      pop.append((w,x))
    print('initial pop : ',pop)
    for gen in range(m):
      children_w = crossover_mutation(omega_set, s-len(pop))
      children_w_x = [(w, find_optimal_x(w,map).tolist()) for w in children_w]
      pop = pop + children_w_x
      print('Generation : ',gen, 'Population : ',pop)
      X_pop = [x for (w,x) in pop]
      mmr, x_star, w_star = minimax_regret(omega_set, X_pop)
      lastest_x_star = x_star
      no_improvement = 0
      while mmr > threshold:
        print('DM Loop : ', dm_counter)
        dm_counter += 1
        (w1, x1), (w2, x2) = query(pop,theta_x)
        if w1 is None:
          break
        xa, xb, ya, yb = ask_DM(x1,x2,w_star)
        theta_x.append((xa, xb))
        omega_set, linear_constraints = solve_omega_set(ya, yb, omega_set, linear_constraints)
        mmr, x_star, w_star = minimax_regret(omega_set, X_pop)
        if lastest_x_star == x_star:
          no_improvement += 1
        else:
          no_improvement = 0
        lastest_x_star = x_star
        if no_improvement > no_improvement_threshold:
          break
      X_pop, pop = selection(x_star, pop, k)
    return x_star, dm_counter, time.time() - start_time, mmr, w_star


# RIGA-S

In [13]:
def selection_riga_s(x, X_pop, k):
    dist_array = [get_tuple_distance(x,p) for p in X_pop]
    sort_index = np.argsort(dist_array).tolist()
    selected_x = [X_pop[i] for i in sort_index[0:k]]
    return selected_x
  
def query_riga_s(X_pop, theta_x):
    c = 0
    t = 1000
    while True:
      x1, x2 = random.sample(X_pop,2)
      if (x1, x2) not in theta_x and (x2, x1) not in theta_x and x1 != x2:
        break
      if c >= t:
        return None, None
      c+=1
    #pop_non_dup = [p for p in pop if p[0] not in big_o_x]
    return x1, x2
# omega_linear_constraints = [[coefficient of w1,w2,w3, bias]]
# cw1 * w1 + cw2 * w2 + ... + cwn * wn + bias <= 0
# only allow 3 objective max

def riga_s_search(map, threshold = 0, n_objectives = 3, s=4, k=2, m=2, no_improvement_threshold=20):
    start_time = time.time()
    dm_counter = 0
    theta_x = []
    omega_set = polyhedral(n_objectives)
    linear_constraints = initialize_linear_constraints(omega_set)
    X_pop = []
    for w in omega_set:
      x = find_optimal_x(w,map).tolist()
      X_pop.append(x)
    print('initial pop : ',X_pop)
    for gen in range(m):
      children_x = crossover_mutation(X_pop, s-len(X_pop),c_type='solution')
      X_pop = X_pop + children_x
      print('Generation : ',gen, 'Population : ',X_pop)
      mmr, x_star, w_star = minimax_regret(omega_set, X_pop)
      lastest_x_star = x_star
      no_improvement = 0
      while mmr > threshold:
        print('-----------------------------------------------')
        print('DM Loop : ', dm_counter)
        dm_counter += 1
        x1, x2 = query_riga_s(X_pop,theta_x)
        if x1 is None:
          break
        xa, xb, ya, yb = ask_DM(x1,x2,w_star)
        theta_x.append((xa, xb))
        omega_set, linear_constraints = solve_omega_set(ya, yb, omega_set, linear_constraints)
        #print('After adjust omega : ', omega_set)
        mmr, x_star, w_star = minimax_regret(omega_set, X_pop)
        if lastest_x_star == x_star:
          no_improvement += 1
        else:
          no_improvement = 0
        lastest_x_star = x_star
        if no_improvement > no_improvement_threshold:
          break
      X_pop = selection_riga_s(x_star, X_pop, k)
    return x_star, dm_counter, time.time() - start_time, mmr, w_star


# IEEP Algorithm

In [14]:
from shapely.geometry import LineString, Polygon, Point
import time
def query_ieep(n_cities, big_o_x):
    c = 0
    t = 1000
    while True:
      x1, x2 = [0]+np.random.permutation(list(range(1,n_cities))).tolist(), [0]+np.random.permutation(list(range(1,n_cities))).tolist()
      if (x1, x2) not in big_o_x and (x2, x1) not in big_o_x and x1 != x2:
        break
      if c >= t:
        return None, None
      c+=1
    return x1, x2

def ieep_search(map, n_cities, threshold = 0, n_objectives = 3, no_improvement_threshold=10):
    start_time = time.time()
    dm_counter = 0
    omega_set = polyhedral(n_objectives)
    linear_constraints = initialize_linear_constraints(omega_set)
    theta_x = []
    X_pop = []
    for w in omega_set:
      x = find_optimal_x(w,map).tolist()
      #pop.append((w,x))
      X_pop.append(x)
    mmr, x_star, w_star = minimax_regret(omega_set, X_pop)
    lastest_x_star = x_star
    no_improvement = 0

    while mmr > threshold:
      print('DM Loop : ', dm_counter)
      dm_counter += 1
      x1, x2 = query_ieep(n_cities, theta_x)
      X_pop.append(x1)
      X_pop.append(x2)
      _, _, ya, yb = ask_DM(x1,x2,w_star)
      omega_set, linear_constraints = solve_omega_set(ya, yb, omega_set, linear_constraints)
      mmr, x_star, w_star = minimax_regret(omega_set, X_pop)
      print('Omega set : ', omega_set)
      
      if lastest_x_star == x_star:
        no_improvement += 1
      else:
        no_improvement = 0
      lastest_x_star = x_star
      if no_improvement > no_improvement_threshold:
        break
    return x_star, dm_counter, time.time() - start_time, mmr, w_star


# NSGA-II

In [15]:
def search_NSGA2(map):
  start_time = time.time()
  res = find_optimal_result([1,1,1],map)
  x_star = res.X
  obj_value = [tuple(f) for f in res.F]
  return x_star, time.time() - start_time, obj_value

# Evaluation

## Generate Pareto Front

In [16]:
import pygmo as pg
import math

def get_hypervolume(test_y, ref_point):
  hv = pg.hypervolume(test_y).compute(ref_point)
  return round(hv,2)
  
def evaluate(test_y, pareto_front, ref_point):
  test_hv = get_hypervolume(test_y, ref_point)
  pareto_hv = get_hypervolume(pareto_front, ref_point)
  return round(test_hv/pareto_hv,2)

In [17]:
## This is a function to find the pareto optimal and pareto front for the evaluation

# Very slow for many datapoints.  Fastest for many costs, most readable
def is_pareto_efficient_dumb(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        is_efficient[i] = np.all(np.any(costs[:i]>c, axis=1)) and np.all(np.any(costs[i+1:]>c, axis=1))
    return is_efficient


# Fairly fast for many datapoints, less fast for many costs, somewhat readable
def is_pareto_efficient_simple(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient]<c, axis=1)  # Keep any point with a lower cost
            is_efficient[i] = True  # And keep self
    return is_efficient


# Faster than is_pareto_efficient_simple, but less readable.
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

In [18]:
from itertools import permutations
all_possible_solution = [[0]+list(p) for p in permutations(list(range(1,n_cities)))]
obj_list = []
for x in all_possible_solution:
  obj_list.append(list(objective(x)))
pareto_front_idx = is_pareto_efficient(np.array(obj_list))
pareto_front = np.array(obj_list)[pareto_front_idx]

ref_point = np.ones(n_objectives)*((n_cities-1) * grid_size * math.sqrt(2))

## RIGA Evaluation

In [19]:
delta = 0
n_pop = 30
n_selection = 5
n_generation = 15

RIGA_best_x, n_queries, runtime, minimax_regret_value, best_w = riga_search(map = edges_map, threshold = delta, n_objectives = n_objectives, s=n_pop, k=n_selection, m=n_generation)
print('Best Path: ', RIGA_best_x)
print('Number of queries: ', n_queries)
print('Time: ', runtime)
print('MMR: ', minimax_regret_value)
print('Objective Values:',objective(RIGA_best_x))
print('Preference Value: ', scalarizer(best_w, objective(RIGA_best_x)))
RIGA_hv = evaluate([objective(RIGA_best_x)], pareto_front, ref_point)
print('RIGA Hyper Value: ', RIGA_hv)

initial pop :  [([1, 0, 0], [0, 2, 1]), ([0, 1, 0], [0, 2, 1]), ([0, 0, 1], [0, 2, 1])]
Generation :  0 Population :  [([1, 0, 0], [0, 2, 1]), ([0, 1, 0], [0, 2, 1]), ([0, 0, 1], [0, 2, 1]), ((0.0, 1.0, 0.0), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((1.0, 0.0, 0.0), [0, 2, 1]), ((1.0, 0.0, 0.0), [0, 2, 1]), ((0.0, 1.0, 0.0), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((0.5, 0.5, 0.0), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((0.5, 0.0, 0.5), [0, 2, 1]), ((0.3333333333333333, 0.3333333333333333, 0.3333333333333333), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((0.3333333333333333, 0.3333333333333333, 0.3333333333333333), [0, 2, 1]), ((0.5, 0.5, 0.0), [0, 2, 1]), ((0.3333333333333333, 0.3333333333333333, 0.3333333333333333), [0, 2, 1]), ((0.3333333333333333, 0.3333333333333333, 0.3333333333333333), [0, 2, 1]), ((0.5, 0.0, 0.5), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((1.0, 0.0, 0.0), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((0.0, 0.5, 0.5), [0, 2, 1]), ((0.3333333333333333, 0.33333

## RIGA-S

In [20]:
delta = 0
n_pop = 30
n_selection = 5
n_generation = 15

RIGAs_best_x, n_queries, runtime, minimax_regret_value, best_w = riga_s_search(map = edges_map, threshold = delta, n_objectives = n_objectives, s=n_pop, k=n_selection, m=n_generation)
print('Best Path: ', RIGAs_best_x)
print('Number of queries: ', n_queries)
print('Time: ', runtime)
print('MMR: ', minimax_regret_value)
print('Objective Values:',objective(RIGAs_best_x))
print('Preference Value: ', scalarizer(best_w, objective(RIGAs_best_x)))
RIGAs_hv = evaluate([objective(RIGAs_best_x)], pareto_front, ref_point)
print('RIGA Hyper Value: ', RIGAs_hv)

initial pop :  [[0, 2, 1], [0, 2, 1], [0, 2, 1]]
Generation :  0 Population :  [[0, 2, 1], [0, 2, 1], [0, 2, 1], [1, 0, 2], [0, 2, 1], [1, 0, 2], [0, 2, 1], [1, 0, 2], [2, 1, 0], [2, 1, 0], [2, 0, 1], [0, 1, 2], [2, 0, 1], [0, 2, 1], [2, 1, 0], [1, 0, 2], [1, 2, 0], [1, 0, 2], [0, 1, 2], [2, 1, 0], [0, 1, 2], [0, 2, 1], [2, 1, 0], [0, 1, 2], [2, 1, 0], [0, 1, 2], [2, 1, 0], [2, 1, 0], [2, 1, 0], [0, 2, 1]]
MMR :  14.968769216094287 X* :  [2, 1, 0]
W* :  [0, 1, 0]
-----------------------------------------------
DM Loop :  0
MMR :  14.968769216094287 X* :  [2, 1, 0]
W* :  [0, 1, 0]
-----------------------------------------------
DM Loop :  1
MMR :  14.968769216094287 X* :  [2, 1, 0]
W* :  [0, 1, 0]
-----------------------------------------------
DM Loop :  2
MMR :  14.968769216094287 X* :  [2, 1, 0]
W* :  [0, 1, 0]
-----------------------------------------------
DM Loop :  3
MMR :  14.968769216094287 X* :  [2, 1, 0]
W* :  [0, 1, 0]
-----------------------------------------------
DM Loop 

## IEEP Evaluation

In [21]:
delta = 0

IEEP_best_x, n_queries, runtime, minimax_regret_value, best_w = ieep_search(edges_map, n_cities, threshold = delta, no_improvement_threshold=20)
print('Best Path: ', IEEP_best_x)
print('Number of queries: ', n_queries)
print('Time: ', runtime)
print('MMR: ', minimax_regret_value)
print('Objective Values:',objective(IEEP_best_x))
print('Preference Value: ', scalarizer(best_w, objective(IEEP_best_x)))
IEEP_hv = evaluate([objective(IEEP_best_x)], pareto_front, ref_point)
print('IEEP Hyper Value: ', IEEP_hv)

MMR :  0.0 X* :  [0, 2, 1]
W* :  [1, 0, 0]
Best Path:  [0, 2, 1]
Number of queries:  0
Time:  1.3822581768035889
MMR:  0.0
Objective Values: (125.54441665068164, 80.65137453819396, 139.64599284788483)
Preference Value:  125.54441665068164
IEEP Hyper Value:  0.61


## NSGA2 Evaluation

In [22]:
NSGA2_best_x, runtime, obj_value = search_NSGA2(edges_map)
print('Best Path: ', NSGA2_best_x)
print('Time: ', runtime)
print('Objective Values:',obj_value)

NSGA2_hv = evaluate(obj_value, pareto_front, ref_point)
print('NSGA2 Hyper Value: ', NSGA2_hv)

Best Path:  [[0 2 1]
 [0 1 2]]
Time:  0.47947239875793457
Objective Values: [(184.8316680779851, 107.66530961159452, 152.1091709643161), (184.8316680779851, 107.66530961159452, 152.1091709643161)]
NSGA2 Hyper Value:  0.3
